the initial data is already cleared according to their length and correlation

In [1]:
%reset -f

import pandas as pd
import os
import re
import numpy as np
from six.moves import xrange
import math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
def calc_force(df):
    df['force'] = df['Fx']**2 + df['Fy']**2 + df['Fz']**2
    df['force'] = df['force'].pow(1/2)
    return df

def calc_velocity(df):
    df['velocity'] = df['Lx']**2 + df['Ly']**2 + df['Lz']
    df['velocity'] = df['velocity'].pow(1/2)
    return df

### load precleared data

In [3]:
all_data = pd.read_csv('../data/new_all_data.csv')
all_data = calc_force(all_data)
all_data = calc_velocity(all_data)
all_data = all_data[['day', 'uid', 'exc_num', 'exc_times', 'velocity', 'force', 'front']]
all_data.head()

,day,uid,exc_num,exc_times,velocity,force,front
0,1,7,1.4,3,0.258776,35.449781,0.178614
1,1,7,1.4,3,0.258776,35.449781,0.178614
2,1,7,1.4,3,0.258776,35.449781,0.178614
3,1,7,1.4,3,0.275260,34.657235,0.178936
4,1,7,1.4,3,0.275260,34.657235,0.178936


### clear outlier that recorded in file

In [6]:
outliers = []

with open('../data/figure/deviation_curves_outlier.csv') as f:
    for line in f.readlines():
        tmp = line.split('\n')[0].split(' ')
       
        outliers.append([float(x) for x in tmp])
        
groups = all_data.set_index(['day', 'exc_num', 'exc_times', 'uid'])
groups = groups.groupby(by=groups.index)

new_data = pd.DataFrame()

for index, group in groups:
    if len(group)<10:
        continue
    day, exc_num, exc_times, uid = index
    exc_num = round(exc_num, 1)
    if [day, exc_num, exc_times, uid] in outliers:
        pass
#         tmp = group.reset_index()
#         sns.set_style('whitegrid')
#         f, ax= plt.subplots(figsize = (14, 10))
#         ax = sns.lineplot(x=tmp.index, y="front", data=tmp)
#         ax.set_title(index) 
    else:
        curr = group.reset_index()
        new_data = pd.concat([new_data, curr], axis=0)

In [7]:
new_data.head()

,day,exc_num,exc_times,uid,velocity,force,front
0,1,1.1,1,4,0.021297,18.561272,0.011647
1,1,1.1,1,4,0.021297,18.561272,0.011647
2,1,1.1,1,4,0.023768,23.049513,0.011971
3,1,1.1,1,4,0.023768,23.049513,0.012333
4,1,1.1,1,4,0.031793,32.181287,0.012333


In [8]:
new_data.to_csv('../data/step1_clear_data.csv')